In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import scipy.io
import numpy as np
data= scipy.io.loadmat('/content/drive/MyDrive/Data/HYS/Indian_pines_corrected.mat')['indian_pines_corrected']
gt= scipy.io.loadmat('/content/drive/MyDrive/Data/HYS/Indian_pines_gt.mat')['indian_pines_gt']

# **evaluate computational complexity**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data.reshape(-1, data.shape[2])).reshape(data.shape)
data_flattened = data_normalized.reshape(-1, data.shape[2])
gt_flattened = gt.ravel()
mask = gt_flattened > 0
data_flattened = data_flattened[mask]
gt_flattened = gt_flattened[mask]
X_train, X_test, y_train, y_test = train_test_split(data_flattened, gt_flattened, test_size=0.3, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data.reshape(-1, data.shape[2])).reshape(data.shape)
data_flattened = data_normalized.reshape(-1, data.shape[2])
gt_flattened = gt.ravel()
mask = gt_flattened > 0
data_flattened = data_flattened[mask]
gt_flattened = gt_flattened[mask]
X_train, X_test, y_train, y_test = train_test_split(data_flattened, gt_flattened, test_size=0.3, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout
from tensorflow.keras.utils import to_categorical

# Convert labels to categorical
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)
num_classes = y_train_categorical.shape[1]

# Build the 1D CNN model
def build_1d_cnn(input_shape, num_classes):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model_1d = build_1d_cnn(input_shape=(200, 1), num_classes=num_classes)
X_train_1d = X_train.reshape(-1, 200, 1)
X_test_1d = X_test.reshape(-1, 200, 1)
history_1d = model_1d.fit(X_train_1d, y_train_categorical, epochs=10, batch_size=32, validation_data=(X_test_1d, y_test_categorical), verbose=1)


Epoch 1/10
225/225 [==============================] - 12s 6ms/step - loss: 1.4321 - accuracy: 0.4968 - val_loss: 1.1400 - val_accuracy: 0.5545
Epoch 2/10
225/225 [==============================] - 1s 5ms/step - loss: 1.0987 - accuracy: 0.6096 - val_loss: 0.9285 - val_accuracy: 0.6712
Epoch 3/10
225/225 [==============================] - 2s 7ms/step - loss: 0.9588 - accuracy: 0.6558 - val_loss: 0.8105 - val_accuracy: 0.7327
Epoch 4/10
225/225 [==============================] - 1s 6ms/step - loss: 0.8585 - accuracy: 0.6809 - val_loss: 0.8058 - val_accuracy: 0.6998
Epoch 5/10
225/225 [==============================] - 1s 6ms/step - loss: 0.7904 - accuracy: 0.7078 - val_loss: 0.6756 - val_accuracy: 0.7629
Epoch 6/10
225/225 [==============================] - 1s 6ms/step - loss: 0.7462 - accuracy: 0.7222 - val_loss: 0.6352 - val_accuracy: 0.7740
Epoch 7/10
225/225 [==============================] - 1s 5ms/step - loss: 0.7037 - accuracy: 0.7414 - val_loss: 0.6632 - val_accuracy: 0.7509
Epoch

In [ ]:
def extract_patches(data, gt, window_size=9):
    margin = window_size // 2
    height, width = gt.shape
    patches = []
    labels = []

    for i in range(margin, height - margin):
        for j in range(margin, width - margin):
            if gt[i, j] > 0:
                patch = data[i-margin:i+margin+1, j-margin:j+margin+1]
                patches.append(patch)
                labels.append(gt[i, j])

    return np.array(patches), np.array(labels)

# Extract patches
X_patches, y_patches = extract_patches(data_normalized, gt)
y_patches_categorical = to_categorical(y_patches)
X_train_2d, X_test_2d, y_train_2d, y_test_2d = train_test_split(X_patches, y_patches_categorical, test_size=0.3, random_state=42)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
def build_2d_cnn(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    # Possibly remove this pooling layer if dimensionality is still an issue
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Build and train the adjusted 2D CNN
model_2d = build_2d_cnn(input_shape=(9, 9, 200), num_classes=num_classes)
history_2d = model_2d.fit(X_train_2d, y_train_2d, epochs=10, batch_size=32, validation_data=(X_test_2d, y_test_2d), verbose=1)

Epoch 1/10
216/216 [==============================] - 4s 11ms/step - loss: 1.3387 - accuracy: 0.5517 - val_loss: 0.6760 - val_accuracy: 0.7387
Epoch 2/10
216/216 [==============================] - 1s 6ms/step - loss: 0.6477 - accuracy: 0.7680 - val_loss: 0.3342 - val_accuracy: 0.8867
Epoch 3/10
216/216 [==============================] - 1s 6ms/step - loss: 0.4002 - accuracy: 0.8571 - val_loss: 0.3436 - val_accuracy: 0.8671
Epoch 4/10
216/216 [==============================] - 2s 8ms/step - loss: 0.2555 - accuracy: 0.9104 - val_loss: 0.2187 - val_accuracy: 0.9172
Epoch 5/10
216/216 [==============================] - 2s 9ms/step - loss: 0.1975 - accuracy: 0.9322 - val_loss: 0.0830 - val_accuracy: 0.9743
Epoch 6/10
216/216 [==============================] - 2s 8ms/step - loss: 0.1143 - accuracy: 0.9610 - val_loss: 0.1200 - val_accuracy: 0.9628
Epoch 7/10
216/216 [==============================] - 2s 7ms/step - loss: 0.1425 - accuracy: 0.9504 - val_loss: 0.1292 - val_accuracy: 0.9567
Epoch

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Evaluate the model to get the accuracy
loss, accuracy = model_2d.evaluate(X_test_2d, y_test_2d, verbose=0)
print(f"Test Accuracy: {accuracy*100:.2f}%")

# Predict the classes
predictions = model_2d.predict(X_test_2d)
predicted_classes = np.argmax(predictions, axis=1)

# Convert one-hot encoded y_test_2d back to class labels
true_classes = np.argmax(y_test_2d, axis=1)

# Calculate precision and recall
report = classification_report(true_classes, predicted_classes)
print(report)


Test Accuracy: 99.12%
93/93 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

           1       1.00      0.88      0.93         8
           2       0.96      1.00      0.98       436
           3       1.00      1.00      1.00       224
           4       1.00      0.97      0.99        70
           5       1.00      0.99      1.00       134
           6       1.00      1.00      1.00       194
           7       1.00      1.00      1.00         4
           8       1.00      1.00      1.00       130
           9       1.00      1.00      1.00         3
          10       1.00      0.99      0.99       298
          11       1.00      0.98      0.99       685
          12       0.99      0.99      0.99       185
          13       1.00      1.00      1.00        67
          14       0.99      0.99      0.99       388
          15       0.97      0.99      0.98       105
          16       0.93      1.00      0.96        27

    a

In [ ]:
def extract_3d_patches(data, gt, window_size=9):
    margin = window_size // 2
    height, width = gt.shape
    patches = []
    labels = []

    for i in range(margin, height - margin):
        for j in range(margin, width - margin):
            if gt[i, j] > 0:
                patch = data[i-margin:i+margin+1, j-margin:j+margin+1, :]
                patches.append(patch)
                labels.append(gt[i, j])

    return np.array(patches), np.array(labels)

# Extract 9x9x200 patches
X_patches_3d, y_patches_3d = extract_3d_patches(data_normalized, gt)
y_patches_categorical_3d = to_categorical(y_patches_3d)
X_train_3d, X_test_3d, y_train_3d, y_test_3d = train_test_split(X_patches_3d, y_patches_categorical_3d, test_size=0.3, random_state=42)

# Adjusted 3D CNN model
def build_3d_cnn(input_shape, num_classes):
    model = Sequential()
    model.add(Conv3D(32, kernel_size=(3, 3, 7), activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Conv3D(64, kernel_size=(3, 3, 5), activation='relu', padding='same'))
    # Possibly remove this pooling layer if dimensionality is still an issue
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Build and train the adjusted 3D CNN
model_3d = build_3d_cnn(input_shape=(9, 9, 200, 1), num_classes=num_classes)
history_3d = model_3d.fit(X_train_3d, y_train_3d, epochs=50, batch_size=32, validation_data=(X_test_3d, y_test_3d), verbose=1)


Epoch 1/50
216/216 [==============================] - 8s 32ms/step - loss: 1.2329 - accuracy: 0.5864 - val_loss: 0.6310 - val_accuracy: 0.7681
Epoch 2/50
216/216 [==============================] - 6s 28ms/step - loss: 0.6026 - accuracy: 0.7812 - val_loss: 0.3581 - val_accuracy: 0.8627
Epoch 3/50
216/216 [==============================] - 6s 27ms/step - loss: 0.3530 - accuracy: 0.8699 - val_loss: 0.1725 - val_accuracy: 0.9402
Epoch 4/50
216/216 [==============================] - 6s 30ms/step - loss: 0.2362 - accuracy: 0.9116 - val_loss: 0.1001 - val_accuracy: 0.9692
Epoch 5/50
216/216 [==============================] - 6s 27ms/step - loss: 0.1604 - accuracy: 0.9391 - val_loss: 0.0593 - val_accuracy: 0.9821
Epoch 6/50
216/216 [==============================] - 6s 28ms/step - loss: 0.1432 - accuracy: 0.9493 - val_loss: 0.0471 - val_accuracy: 0.9848
Epoch 7/50
216/216 [==============================] - 6s 30ms/step - loss: 0.1233 - accuracy: 0.9555 - val_loss: 0.0430 - val_accuracy: 0.9899

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Predict the classes using the 3D CNN model
predictions_3d = model_3d.predict(X_test_3d)
predicted_classes_3d = np.argmax(predictions_3d, axis=1)

# Convert one-hot encoded y_test_3d back to class labels
true_classes_3d = np.argmax(y_test_3d, axis=1)

# Calculate accuracy
accuracy_3d = np.mean(predicted_classes_3d == true_classes_3d)
print(f"Test Accuracy: {accuracy_3d*100:.2f}%")

# Calculate precision and recall
report_3d = classification_report(true_classes_3d, predicted_classes_3d)
print(report_3d)


93/93 [==============================] - 1s 10ms/step
Test Accuracy: 99.76%
              precision    recall  f1-score   support

           1       1.00      0.88      0.93         8
           2       1.00      0.99      0.99       436
           3       1.00      1.00      1.00       224
           4       1.00      1.00      1.00        70
           5       1.00      1.00      1.00       134
           6       1.00      1.00      1.00       194
           7       1.00      1.00      1.00         4
           8       1.00      1.00      1.00       130
           9       1.00      1.00      1.00         3
          10       1.00      1.00      1.00       298
          11       0.99      1.00      1.00       685
          12       1.00      1.00      1.00       185
          13       1.00      1.00      1.00        67
          14       1.00      1.00      1.00       388
          15       1.00      1.00      1.00       105
          16       1.00      1.00      1.00        27

    